# PS2: Selection of an Uncorrelated Tangent Portfolio
Fill me in.

## Problem statement
* Task 1: Compute the log return array and covariance matrix for a collection of `N = 10` stocks that you select from the historical dataset 
* Task 2: Compute the efficient frontier for your collection of `N = 10` stocks
* Task 3: Compute the capital allocation line connecting the risk-free asset and the tangent portfolio
* Task 4: Use your estimated value for the risk aversion parameter `A` to estimate your investment allocation along the capital allocation line

## Setup
The computations in this lab (or example) are enabled by the [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) and several external `Julia` packages. To load the required packages and any custom codes the teaching team has developed to work with these packages, we [include](https://docs.julialang.org/en/v1/manual/code-loading/) the `Include.jl` file):

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-PS2-AssetSelectionProblem-Fall-2023/Manifest.toml`


### Packages
`Include.jl` loads several external packages that we will use for our excercise:
* [DataFrames.jl](https://dataframes.juliadata.org/stable/) provides a set of tools for working with tabular data in [Julia](https://julialang.org). Its design and functionality are similar to those of [Pandas (in Python)](https://pandas.pydata.org) and [data.frame, data.table and dplyr (in R)](https://dplyr.tidyverse.org), making it a great general purpose data science tool.
* [Plots.jl](https://docs.juliaplots.org/stable/) is a plotting library in [Julia](https://julialang.org). [Plots.jl](https://docs.juliaplots.org/stable/) exports the `plot(...)` function which we use to visualize data.
* [VLDecisionsPackage.jl](https://github.com/varnerlab/VLDecisionsPackage.jl.git) holds the codes and types for most of the computation used in the problem set.

### Types
The `MyMarkowitzRiskyAssetOnlyPortfiolioChoiceProblem` type, which encodes data required to compute the efficient frontier for a collection of $N$ risky assets only, has the fields:
* `Σ::Array{Float64,2}` is the $N\times{N}$ covariance matrix, with entries $cov(r_{i},r_{j})$
* `μ::Array{Float64,1}` is the $N\times{1}$ vector of expected return values with elements $\mathbb{E}(r_{i})$
* `bounds::Array{Float64,2}` is an $N\times{2}$ array that holds the upper and lower bound for the asset allocation. Each row corresponds to an asset, while the first column is the lower bound, and the second column is the upper bound on the asset allocation fraction.
* `R::Float64` denotes the lower bound on the desired return
* `initial::Array{Float64,1}` is the $N\times{1}$ array of initial guesses

The `MyMarkowitzRiskyRiskFreePortfiolioChoiceProblem`, which encodes data required to compute the optimal allocation for a portfolio containing $N$ risky assets and a single risk-free asset, has the fields:
* `Σ::Array{Float64,2}` is the $N\times{N}$ covariance matrix, with entries $cov(r_{i},r_{j})$
* `μ::Array{Float64,1}` is the $N\times{1}$ vector of expected return values, with elements $\mathbb{E}(r_{i})$
* `bounds::Array{Float64,2}` is an $N\times{2}$ array that holds the upper and lower bound for the asset allocation. Each row corresponds to an asset, while the first column is the lower bound, and the second column is the upper bound on the asset allocation fraction.
* `R::Float64` denotes the lower bound on the desired return
* `initial::Array{Float64,1}` is the $N\times{1}$ array of initial guesses
* `risk_free_rate::Float64` is the risk-free rate of return, e.g., the rate of return on a [10-year US Treasury Note](https://www.cnbc.com/bonds/)

#### Functions
* The `MyPortfolioDataSet()` function loads the risky asset price dataset, and returns it as a [DataFrame.jl](https://dataframes.juliadata.org/stable/) instance. 
* The ` MyFirmMappingDataSet()` function load the firm mapping, and returns it as a [DataFrame.jl](https://dataframes.juliadata.org/stable/) instance.
* The `build(...)` is a factory method that takes the type of object that you wish to construct, and the parameters required to construct the object, e.g., the fields of the `MyMarkowitzRiskyAssetOnlyPortfiolioChoiceProblem` or `MyMarkowitzRiskyRiskFreePortfiolioChoiceProblem` types.
* The `solve(...)` method takes a `problem` type, e.g., an instance of `MyMarkowitzRiskyRiskFreePortfiolioChoiceProblem` and returns the solution to the problem as a [Julia Dictionary](https://docs.julialang.org/en/v1/base/collections/#Dictionaries) instance.

### Load historical dataset
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) for the past five-trading years (a maximum of `1256` data points per firm). However, not all the firms in the `dataset` have the maximum number of trading days, i.e., some firms are missing information for various reasons; perhaps they were acquired, merged, or delisted, etc. We will exclude these firms from the `dataset`.

We load the price `dataset` by calling the `MyPortfolioDataSet()` function:

In [2]:
dataset = MyPortfolioDataSet() |> x-> x["dataset"];

The all keys array holds the list of firm indexes in the dataset that have complete data, i.e., all `1256` data values: 

In [3]:
all_firms = keys(dataset) |> collect |> sort;

However, while it is sometimes convenient to work with the data using the `firm_index`, often we specify the [ticker symbol](https://en.wikipedia.org/wiki/Ticker_symbol#:~:text=A%20ticker%20symbol%20or%20stock,on%20a%20particular%20stock%20market.) instead. To facilitate this, let's load a mapping between the `firm_index` and the ticker symbols using the `MyFirmMappingDataSet()` function. We store this mapping in the `firm_mapping_df` variable, which is of type `DataFrame`:

In [4]:
firm_mapping_df = MyFirmMappingDataSet()

Row,Symbol,Name,Sector
,String7,String,String31
1,MMM,3M,Industrials
2,AOS,A. O. Smith,Industrials
3,ABT,Abbott Laboratories,Health Care
4,ABBV,AbbVie,Health Care
5,ABMD,Abiomed,Health Care
6,ACN,Accenture,Information Technology
7,ATVI,Activision Blizzard,Communication Services
8,ADM,ADM,Consumer Staples
9,ADBE,Adobe,Information Technology


To interconvert between the `firm_index` and `ticker` representation of the firms in the dataset, we create a `ticker_lookup_dictionary` which holds the ticker symbols as the keys and the `firm_index` as the value:

In [5]:
ticker_lookup_dictionary = Dict{String,Int64}();
number_of_map_elements = nrow(firm_mapping_df);
for i ∈ 1:number_of_map_elements
    ticker = firm_mapping_df[i,:Symbol];    
    ticker_lookup_dictionary[ticker] = i;
end

For example, [Apple](https://www.apple.com) has the `ticker = AAPL`, in the dataset that is `firm_index`:

In [6]:
ticker_lookup_dictionary["AAPL"]

46

### Compute the covariance and expected yield for all firms in the dataset
The expected return $\mathbb{E}(r_{i})$ and covariance matrix $\Sigma$ will be used in your calculations, so we'll provide values for both of these items for the entire data set (all `N = 458` companies), and then you can pick out which forms you are interested in. 

First, we compute the expected (annualized) log return by passing the `dataset` and the entire list of firms we have in the dataset (held in the $N\times{1}$ `all_firms` array) to the `log_return_matrix(...)` method. The result is stored in the `all_firms_return_matrix` variable, a $T-1\times{N}$ array of log return values. Each row of `all_firms_return_matrix` corresponds to a time-value, while each column corresponds to a firm:

In [7]:
all_firms_return_matrix = log_return_matrix(dataset, all_firms; Δt = (1/252));

Next, we estimate the `covariance_matrix` from the `all_firms_return_matrix` using the `cov(...)` function, which is exported by the [Statistics.jl package](https://docs.julialang.org/en/v1/stdlib/Statistics/). We store the $N\times{N}$ covariance matrix in the $\Sigma$ variable:

In [8]:
Σ = cov(all_firms_return_matrix) |> x-> x*(1/100);

Finally, we estimate the expected return for each firm from the `all_firms_return_matrix` using the `mean(...)` function, which is exported by the [Statistics.jl package](https://docs.julialang.org/en/v1/stdlib/Statistics/). We set the $N\times{1}$ vector of expected values, i.e., $\mathbb{E}(r_{1}),\dotsc,\mathbb{E}(r_{N})$ in the variable $\mu$:

In [9]:
μ = mean(all_firms_return_matrix, dims=1) |> vec |> x-> exp.(x) .- 1;

### Set constants used in the calculations
Lastly, we set some constant values that are used in the computation of the efficient frontier and the capital allocation line:

In [10]:
number_of_firms = 10;
initial = zeros(number_of_firms);
initial[1] = 1.0;
bounds = zeros(number_of_firms,2);
bounds[:,2] .= 1.0;
risk_free_rate = 0.045
number_of_points = 20;

## Task 1: Compute the log return array and covariance matrix for your selection of stocks

Specify five `ticker symbols`, e.g., `AAPL`, `MSFT`, etc that you are interested the `my_list_of_tickers` array. We'll hold then put the indexes of your five firms in the `my_list_of_firm_ids` array:

In [17]:
my_list_of_tickers = ["ADBE","ZTS","AAPL"];
my_list_of_firm_ids = Array{Int64,1}();
for ticker ∈ my_list_of_tickers
    firm_index = ticker_lookup_dictionary[ticker];    
    push!(my_list_of_firm_ids, firm_index)
end

Next, for each the firms you selected, find the lowest correlation partner firm by analyzing the entries of the covariance matrix. Add the index of the partner firm to the `my_list_of_firm_ids` array using the `push!(...)` function:

In [15]:
# fill me in here

Now that you have selected the firms in your risky portfolio, we'll compute the firm-specific expected return vector and covariance matrix. We'll store these in the $\hat{\mu}$ and $\hat{\Sigma}$ variables, respectively. First, we construct the $\hat{\mu}$ vector:

In [19]:
μ̂ = Array{Float64,1}();
for i ∈ eachindex(my_list_of_firm_ids)
    firm_index = findfirst(x->x==my_list_of_firm_ids[i], all_firms)
    push!(μ̂, μ[firm_index])
end

and then the covariance $\hat{\Sigma}$ matrix:

## Task 2: Compute the efficient frontier for your collection of `N = 10` stocks
Fill me in

## Task 3: Compute the capital allocation line for your collection of `N = 10` stocks and risk free asset
Fill me in

## Task 4: Compute your operating point
Fill me in

## Disclaimer and Risks

__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team.

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance. Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. Such decisions should be based solely on evaluating your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.